<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/output_parsing/lmformatenforcer_regular_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LM Format Enforcer Regular Expression Generation

Generate structured data with [**lm-format-enforcer**](https://github.com/noamgat/lm-format-enforcer) via LlamaIndex.  


With lm-format-enforcer, you can guarantee the output structure is correct by *forcing* the LLM to output desired tokens.  
This is especialy helpful when you are using lower-capacity model (e.g. the current open source models), which otherwise would struggle to generate valid output that fits the desired output schema.

[lm-format-enforcer](https://github.com/noamgat/lm-format-enforcer) supports regular expressions and JSON Schema, this demo focuses on regular expressions. For JSON Schema + Pydantic, see the [sample Pydantic program notebook](https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/output_parsing/lmformatenforcer_pydantic_program.ipynb).

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-llms-llama-cpp

In [ ]:
!pip install llama-index llama-index-program-lmformatenforcer lm-format-enforcer llama-cpp-python

In [ ]:
import lmformatenforcer
import re

from llama_index.program.lmformatenforcer.utils import (
    activate_lm_format_enforcer,
    build_lm_format_enforcer_function,
)

Define output format

In [ ]:
regex = r'"Hello, my name is (?P<name>[a-zA-Z]*)\. I was born in (?P<hometown>[a-zA-Z]*). Nice to meet you!"'

Create the model. We use `LlamaCPP` as the LLM in this demo, but `HuggingFaceLLM` is also supported.

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP

llm = LlamaCPP()

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /mnt/wsl/PHYSICALDRIVE1p3/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.we

Activate the format enforcer and run the LLM get structured output in the desired regular expression format. As long as we are inside the `with activate_lm_format_enforcer(...)` block, the LLM will output the desired format.

If we would have used `lmformatenforcer.JsonSchemaParser` and a JSON schema, we would have gotten JSON output instead.

In [ ]:
regex_parser = lmformatenforcer.RegexParser(regex)
lm_format_enforcer_fn = build_lm_format_enforcer_function(llm, regex_parser)
with activate_lm_format_enforcer(llm, lm_format_enforcer_fn):
    output = llm.complete(
        "Here is a way to present myself, if my name was John and I born in Boston: "
    )


llama_print_timings:        load time =  2709.44 ms
llama_print_timings:      sample time =     7.26 ms /    22 runs   (    0.33 ms per token,  3031.56 tokens per second)
llama_print_timings: prompt eval time =  2709.40 ms /    21 tokens (  129.02 ms per token,     7.75 tokens per second)
llama_print_timings:        eval time =  3047.28 ms /    21 runs   (  145.11 ms per token,     6.89 tokens per second)
llama_print_timings:       total time =  5965.41 ms


The output is a string, according to the regular expression, which we can parse and extract parameters from.

In [ ]:
print(output)
print(re.match(regex, output.text).groupdict())

"Hello, my name is John. I was born in Boston, Nice to meet you!"
{'name': 'John', 'hometown': 'Boston'}
